In [2]:
#   ------------------------------------
#   Libraries
#   ------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from enum import Enum
import random

#   For land/ocean coordinates identification
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import shapely.geometry as sgeom

#   For nice visualizations
import folium
import geopy.distance

In [24]:
#   ------------------------------------
#   Weather Data
#   ------------------------------------
weather_df = pd.read_csv('hurricane_milton.csv')

In [25]:
print(weather_df.shape)

(45936, 19)


In [26]:
weather_df.head()

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
0,2024-10-09 00:00:00,32.0,-88.00,282.47168,23.69192,1011.6150,NaN,1004.5244,0.955292,-5.644577,0.0,0.988942,0.011058,3,1,594.20166,0.0,0.0,0.0
1,2024-10-09 00:00:00,32.0,-87.75,283.12988,23.69192,1011.4625,NaN,1003.0244,1.092987,-5.639694,0.0,0.992065,0.005601,3,0,708.60400,0.0,0.0,0.0
2,2024-10-09 00:00:00,32.0,-87.50,282.69043,24.45950,1011.2700,NaN,1003.5944,0.744354,-5.595749,0.0,0.947906,0.048926,3,1,644.97900,0.0,0.0,0.0
3,2024-10-09 00:00:00,32.0,-87.25,282.68457,24.86184,1011.1325,NaN,1003.2744,-0.144318,-5.181686,0.0,0.947906,0.048926,3,1,660.78370,0.0,0.0,0.0
4,2024-10-09 00:00:00,32.0,-87.00,282.98730,24.73293,1011.0750,NaN,1001.6144,-1.456818,-4.544968,0.0,0.981760,0.018240,3,1,796.74070,0.0,0.0,0.0


In [14]:
#   Rename columns
weather_df = weather_df.rename(columns={
    't2m': 'Temperature', 
    'd2m': 'Dewpoint', 
    'msl': 'Mean Sea Level Pressure', 
    'sst': 'Sea Surface Temperature',
    'sp': 'Surface Pressure',
    'u100': 'East Wind Speed',
    'v100': 'North Wind Speed',
    'crr': 'Convective Rain Rate',
    'cvh': 'High Vegetation Cover',
    'cvl': 'Low Vegetation Cover',
    'tvh': 'Type of High Vegetation',
    'tvl': 'Type of Low Vegetation',
    'z': 'Geopotential',
    'tp': 'Total Precipitation',
    'cp': 'Convective Precipitation',
    'lsp': 'Large Scale Precipitation' })

In [8]:
weather_df.head()

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
0,2024-10-09 00:00:00,32.0,-88.00,282.47168,23.69192,101161.50,NaN,100452.44,0.955292,-5.644577,0.0,0.988942,0.011058,3,1,594.20166,0.0,0.0,0.0
1,2024-10-09 00:00:00,32.0,-87.75,283.12988,23.69192,101146.25,NaN,100302.44,1.092987,-5.639694,0.0,0.992065,0.005601,3,0,708.60400,0.0,0.0,0.0
2,2024-10-09 00:00:00,32.0,-87.50,282.69043,24.45950,101127.00,NaN,100359.44,0.744354,-5.595749,0.0,0.947906,0.048926,3,1,644.97900,0.0,0.0,0.0
3,2024-10-09 00:00:00,32.0,-87.25,282.68457,24.86184,101113.25,NaN,100327.44,-0.144318,-5.181686,0.0,0.947906,0.048926,3,1,660.78370,0.0,0.0,0.0
4,2024-10-09 00:00:00,32.0,-87.00,282.98730,24.73293,101107.50,NaN,100161.44,-1.456818,-4.544968,0.0,0.981760,0.018240,3,1,796.74070,0.0,0.0,0.0


In [ ]:
#   Convert all temperatures to Celsius (currently in Kelvin)
weather_df['Temperature'] = weather_df['Temperature'] - 273.15
weather_df['Sea Surface Temperature'] = weather_df['Sea Surface Temperature'] - 273.15
#   Convert all pressures Pascal (Pa) to Hectopascal (hPa) which is the unit used in meteorology.
weather_df['Mean Sea Level Pressure'] = weather_df['Mean Sea Level Pressure'] / 100
weather_df['Surface Pressure'] = weather_df['Surface Pressure'] / 100

In [10]:
weather_df.head()

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
0,2024-10-09 00:00:00,32.0,-88.00,282.47168,-249.45808,101161.50,NaN,100452.44,0.955292,-5.644577,0.0,0.988942,0.011058,3,1,594.20166,0.0,0.0,0.0
1,2024-10-09 00:00:00,32.0,-87.75,283.12988,-249.45808,101146.25,NaN,100302.44,1.092987,-5.639694,0.0,0.992065,0.005601,3,0,708.60400,0.0,0.0,0.0
2,2024-10-09 00:00:00,32.0,-87.50,282.69043,-248.69050,101127.00,NaN,100359.44,0.744354,-5.595749,0.0,0.947906,0.048926,3,1,644.97900,0.0,0.0,0.0
3,2024-10-09 00:00:00,32.0,-87.25,282.68457,-248.28816,101113.25,NaN,100327.44,-0.144318,-5.181686,0.0,0.947906,0.048926,3,1,660.78370,0.0,0.0,0.0
4,2024-10-09 00:00:00,32.0,-87.00,282.98730,-248.41707,101107.50,NaN,100161.44,-1.456818,-4.544968,0.0,0.981760,0.018240,3,1,796.74070,0.0,0.0,0.0


In [27]:
weather_df['valid_time'] = pd.to_datetime(weather_df['valid_time'])

In [28]:
print(weather_df.dtypes)

valid_time                   datetime64[ns]
latitude                            float64
longitude                           float64
Dewpoint                            float64
Temperature                         float64
Mean Sea Level Pressure             float64
Sea Surface Temperature             float64
Surface Pressure                    float64
East Wind Speed                     float64
North Wind Speed                    float64
Convective Rain Rate                float64
High Vegetation Cover               float64
Low Vegetation Cover                float64
Type of High Vegetation               int64
Type of Low Vegetation                int64
Geopotential                        float64
Total Precipitation                 float64
Convective Precipitation            float64
Large Scale Precipitation           float64
dtype: object


In [ ]:
# Saving empty_df to a CSV file
#weather_df.to_csv('hurricane_milton.csv', index=False)

#print("DataFrame saved to hurricane_milton_clean.csv")

In [ ]:
# Filtering rows where 'new_time' is equal to '2024-10-09 00:20:00'
filtered_df = weather_df[weather_df['valid_time'] == pd.Timestamp('2024-10-09 20:00:00')]
filtered_df.shape

(957, 19)

In [30]:
filtered_df.head()


,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
19140,2024-10-09 20:00:00,32.0,-88.00,287.42065,31.35660,1011.5244,NaN,1004.4869,-3.075729,-4.412247,0.0,0.988942,0.011058,3,1,594.20166,0.0,0.0,0.0
19141,2024-10-09 20:00:00,32.0,-87.75,287.80150,31.09878,1011.4294,NaN,1003.0369,-3.371628,-4.345841,0.0,0.992065,0.005601,3,0,708.60400,0.0,0.0,0.0
19142,2024-10-09 20:00:00,32.0,-87.50,287.31910,31.40542,1011.3119,NaN,1003.6669,-3.588425,-4.281387,0.0,0.947906,0.048926,3,1,644.97900,0.0,0.0,0.0
19143,2024-10-09 20:00:00,32.0,-87.25,286.66480,31.68862,1011.2394,NaN,1003.4069,-3.683151,-4.338028,0.0,0.947906,0.048926,3,1,660.78370,0.0,0.0,0.0
19144,2024-10-09 20:00:00,32.0,-87.00,286.35034,31.51870,1011.2344,NaN,1001.7969,-3.824753,-4.435684,0.0,0.981760,0.018240,3,1,796.74070,0.0,0.0,0.0


In [31]:
filtered_df.tail()

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
20092,2024-10-09 20:00:00,25.0,-81.00,298.71558,27.44253,1001.9494,29.27188,1001.8269,3.226028,20.267440,0.000008,0.0,0.0,0,0,10.318848,0.000757,0.000668,0.000090
20093,2024-10-09 20:00:00,25.0,-80.75,298.55737,27.71010,1002.5619,29.28457,1002.4669,2.446732,19.210800,0.000229,0.0,0.0,0,13,8.033691,0.001308,0.001280,0.000028
20094,2024-10-09 20:00:00,25.0,-80.50,298.57690,28.10270,1003.2569,29.41152,1003.1669,2.454544,18.480331,0.000327,0.0,0.0,0,0,7.826660,0.001207,0.001179,0.000028
20095,2024-10-09 20:00:00,25.0,-80.25,298.81128,28.45034,1003.9544,29.44082,1003.9269,2.438919,17.306503,0.000327,0.0,0.0,0,0,2.369629,0.000759,0.000715,0.000044
20096,2024-10-09 20:00:00,25.0,-80.00,298.85230,28.96987,1004.4394,29.58926,1004.4169,2.063919,16.171738,0.000001,0.0,0.0,0,0,2.111816,0.000336,0.000287,0.000048


In [37]:
lowest_value_SLP = weather_df['Mean Sea Level Pressure'].min()
min_row_SLP = weather_df.loc[weather_df['Mean Sea Level Pressure'] == lowest_value_SLP]
print(lowest_value_SLP)

976.43375


In [39]:
min_row_SLP

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
23649,2024-10-10,27.0,-82.75,298.60645,28.28286,976.43375,28.45547,976.43125,8.970612,4.849442,0.000398,0.0,0.0,0,0,0.604004,0.001584,0.000403,0.001182


In [40]:
lowest_value_SP = weather_df['Surface Pressure'].min()
min_row_SP = weather_df.loc[weather_df['Surface Pressure'] == lowest_value_SP]
print(lowest_value_SP)

976.43125


In [41]:
min_row_SP

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
23649,2024-10-10,27.0,-82.75,298.60645,28.28286,976.43375,28.45547,976.43125,8.970612,4.849442,0.000398,0.0,0.0,0,0,0.604004,0.001584,0.000403,0.001182


In [48]:
# Filtering rows where 'timestamp' is equal to row with lowest pressure in data
filtered_df = weather_df[weather_df['valid_time'] == pd.Timestamp('2024-10-10 00:00:00')]
filtered_df.shape

(957, 19)

In [49]:
filtered_df.head()

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
22968,2024-10-10,32.0,-88.00,288.52637,25.77700,1011.94875,NaN,1004.87125,0.081940,-6.468918,0.0,0.988942,0.011058,3,1,594.20166,0.0,0.0,0.0
22969,2024-10-10,32.0,-87.75,288.57910,25.29067,1011.77875,NaN,1003.35125,-1.102631,-6.047043,0.0,0.992065,0.005601,3,0,708.60400,0.0,0.0,0.0
22970,2024-10-10,32.0,-87.50,288.13574,25.56410,1011.59875,NaN,1003.93125,-2.016693,-5.554855,0.0,0.947906,0.048926,3,1,644.97900,0.0,0.0,0.0
22971,2024-10-10,32.0,-87.25,287.86230,25.67544,1011.51125,NaN,1003.66125,-2.847748,-5.037277,0.0,0.947906,0.048926,3,1,660.78370,0.0,0.0,0.0
22972,2024-10-10,32.0,-87.00,287.87012,25.43910,1011.48625,NaN,1002.02125,-3.415131,-4.591965,0.0,0.981760,0.018240,3,1,796.74070,0.0,0.0,0.0


In [46]:
highest_value_ws = weather_df['North Wind Speed'].max()
min_row_ws = weather_df.loc[weather_df['North Wind Speed'] == highest_value_ws]
print(highest_value_ws)

32.777206


In [47]:
min_row_ws

,valid_time,latitude,longitude,Dewpoint,Temperature,Mean Sea Level Pressure,Sea Surface Temperature,Surface Pressure,East Wind Speed,North Wind Speed,Convective Rain Rate,High Vegetation Cover,Low Vegetation Cover,Type of High Vegetation,Type of Low Vegetation,Geopotential,Total Precipitation,Convective Precipitation,Large Scale Precipitation
19886,2024-10-09 20:00:00,26.5,-83.0,298.40698,28.62808,986.6994,29.23965,986.6669,-1.242722,32.777206,0.000641,0.0,0.0,0,0,3.447754,0.006071,0.003536,0.002534


In [ ]:
# Define the center of Florida
florida_center = (27.9944024, -81.7602544)  # Approximate center of Florida

# Create a Folium map
m = folium.Map(location=florida_center, zoom_start=6)

# Define grid parameters
cell_size_km = 31  # Cell size in kilometers
num_rows = 15  # Number of rows
num_cols = 15  # Number of columns

# Function to calculate new latitude and longitude given a starting point and distance
def move_location(lat, lon, d_lat_miles, d_lon_miles):
    new_lat = geopy.distance.distance(miles=d_lat_miles).destination((lat, lon), 0)[0]  # Move North-South
    new_lon = geopy.distance.distance(miles=d_lon_miles).destination((lat, lon), 90)[1]  # Move East-West
    return new_lat, new_lon

# Function to generate random weather data
def generate_weather_data():
    wind_speed = random.randint(20, 150)  # Wind speed in mph
    pressure = random.randint(900, 1020)  # Pressure in mb
    temperature = random.randint(20, 35)  # Temperature in °C
    humidity = random.randint(60, 100)  # Humidity %
    return wind_speed, pressure, temperature, humidity

# Generate the grid with weather data
start_lat, start_lon = 30, -85  # Starting position (Northwest Florida)
for i in range(num_rows):
    for j in range(num_cols):
        # Compute corners of each 19-mile x 19-mile cell
        top_left = (start_lat, start_lon)
        bottom_right = move_location(start_lat, start_lon, -cell_size_km, cell_size_km)

        # Generate weather data
        wind_speed, pressure, temperature, humidity = generate_weather_data()

        # Determine color based on wind speed (higher wind = more intense color)
        if wind_speed > 120:
            color = "red"
        elif wind_speed > 90:
            color = "orange"
        elif wind_speed > 60:
            color = "yellow"
        else:
            color = "green"

        # Add the grid cell with weather tooltip
        folium.Rectangle(
            bounds=[top_left, bottom_right],
            color=color,
            weight=1,
            fill=True,
            fill_color=color,
            fill_opacity=0.5,
            popup=folium.Popup(
                f"<b>Wind Speed:</b> {wind_speed} mph<br>"
                f"<b>Pressure:</b> {pressure} mb<br>"
                f"<b>Temperature:</b> {temperature}°C<br>"
                f"<b>Humidity:</b> {humidity}%",
                max_width=300
            )
        ).add_to(m)

        # Update longitude for next column
        start_lon = bottom_right[1]

    # Reset longitude and move down in latitude for next row
    start_lon = -85
    start_lat = bottom_right[0]

# Show the map
m
#m.save("hurricane_map.html")
